In [1]:
##pip install scenariogeneration

In [2]:
#from scenariogeneration import xodr
import os
from scenariogeneration import xosc, prettyprint

C:\Users\USER\AppData\Local\Temp\ipykernel_22248\1275792747.py:3: DeprecationWarning: STD_ROADMARK_<RoadMarkType> is deprecated and will be removed in a few releases! Use std_roadmark_<RoadMarkType> functions instead.
  from scenariogeneration import xosc, prettyprint


### ParameterDeclarations (document:xosc.utiles)

In [3]:
egoInit = xosc.Parameter(name="EgoVehicle",parameter_type="string",value="car_white")
egoSpeed = xosc.Parameter(name="EgoSpeed",parameter_type="double",value="60")
egoS = xosc.Parameter(name="Ego_S",parameter_type="double",value="27")

agentInit = xosc.Parameter(name="TargetVehicle",parameter_type="string",value="car_red")
agentSpeed = xosc.Parameter(name="TargetSpeed",parameter_type="double",value="28")
agentS = xosc.Parameter(name="Target_S",parameter_type="double",value="0")

paraList = [egoInit, egoSpeed, egoS, agentInit, agentSpeed, agentS]
paramdec = xosc.ParameterDeclarations()
for i in paraList:
    paramdec.add_parameter(i)

### (Don't modify) CatalogLocations & RoadNetwork (document:xosc.utiles)

In [4]:
# CatalogLocations
catalog = xosc.Catalog()
# catalog.add_catalog("VehicleCatalog", "../Catalogs/Vehicles")
#catalog.add_catalog("ControllerCatalog", "../Catalogs/Controllers")
#catalog.add_catalog("ControllerCatalog", "../../../workspace/src/esmini/resource/xosc/Catalogs/Controllers")
catalog.add_catalog("VehicleCatalog", "./Catalogs/Vehicles")


# RoadNetwork
# road = xosc.RoadNetwork(roadfile="../../xodr/itri/hct_6.xodr")
road = xosc.RoadNetwork(roadfile="hct_6.xodr")

### Entities (document:xosc.Entities)

In [5]:
#construct CatalogReference
egoObject = xosc.CatalogReference(catalogname="VehicleCatalog", entryname="$EgoVehicle") #xosc.utils
agentObject = xosc.CatalogReference(catalogname="VehicleCatalog", entryname="$TargetVehicle") #xosc.utils

#construct ego controller - ROS
# egoControllerProperties = xosc.Properties()
# egoControllerProperties.add_property(name="timeGap", value="1.0")
# egoControllerProperties.add_property(name="mode", value="override")
# egoControllerProperties.add_property(name="setSpeed", value="${$EgoSpeed / 3.6}")
# egoController = xosc.Controller(name="ROSController", properties=egoControllerProperties)

# ACC controller
egoControllerProperties = xosc.Properties()
egoControllerProperties.add_property(name="timeGap", value="1.0")
egoControllerProperties.add_property(name="mode", value="override")
egoControllerProperties.add_property(name="setSpeed", value="${$EgoSpeed / 3.6}")
egoController = xosc.Controller(name="ACCController", properties=egoControllerProperties)

# create entity
egoName = "Ego"
agentName = "Agent"
entities = xosc.Entities()
entities.add_scenario_object(name = egoName, entityobject = egoObject, controller = egoController)
entities.add_scenario_object(name = agentName, entityobject = agentObject)

### Storyboard - Init

In [6]:
# initialzie Ego and Agent
init = xosc.Init() #xosc.storyboard
init.add_init_action(egoName, xosc.TeleportAction(xosc.LanePosition(s = "$Ego_S", offset = 0, lane_id = 1, road_id = 141)))
init.add_init_action(
    egoName,
    xosc.AbsoluteSpeedAction("${$EgoSpeed / 3.6}", xosc.TransitionDynamics(xosc.DynamicsShapes.step, xosc.DynamicsDimension.time, 0)))
init.add_init_action(egoName, xosc.ActivateControllerAction(lateral = "true", longitudinal = "true"))
init.add_init_action(egoName, xosc.AcquirePositionAction(xosc.LanePosition(s = 8, offset = 0, lane_id = -1, road_id = 98)))

init.add_init_action(agentName, xosc.TeleportAction(xosc.LanePosition(s = "$Target_S", offset = 0, lane_id = 1, road_id = 106)))
# init.add_init_action(egoName, xosc.AcquirePositionAction(xosc.LanePosition(s = 8, offset = 0, lane_id = -1, road_id = 106)))




### Storyboard - Event

In [7]:
# 创建一个Trigger-Action，Trigger为仿真一开始即开始，Action为路线规划Route。
egoEvent = xosc.Event("TargetStartWhenEgoCloseToTheJunction", xosc.Priority.overwrite)
egoEvent.add_action("ActivateControllerActionStart", xosc.ActivateControllerAction(lateral = "true", longitudinal = "true"))
egoEvent.add_action("AbsoluteSpeedActionStart", 
                     xosc.AbsoluteSpeedAction("${$TargetSpeed / 3.6}",
                     xosc.TransitionDynamics(xosc.DynamicsShapes.step, xosc.DynamicsDimension.time, 0)))

# target position of Agent
egoEvent.add_action("AcquirePositionActionStart", xosc.AcquirePositionAction(xosc.LanePosition(s = 5, offset = 0, lane_id = -1, road_id = 141)))

# Event start trigger
entityCondition = xosc.ReachPositionCondition(xosc.LanePosition(s = 42, offset = 0, lane_id = 1, road_id = 106),tolerance = 20)
egoEvent.add_trigger(xosc.EntityTrigger(name = "EgoEnteringTheJunction", delay = 0,
                                        conditionedge = xosc.ConditionEdge.rising,
                                        entitycondition = entityCondition, triggerentity = egoName, triggeringrule = "any" ) )


### Storyboard - Story & Maneuver

In [8]:
egoManeuver = xosc.Maneuver("MyManeuver")
egoManeuver.add_event(egoEvent)

sb = xosc.StoryBoard(
    init,
    xosc.EntityTrigger(
"ReachDestinationCondition",
        0,
        xosc.ConditionEdge.rising,
        entitycondition = xosc.ReachPositionCondition(xosc.LanePosition(s = 20, offset = 0, lane_id = 1, road_id = 106),tolerance = 10),
        triggerentity = agentName,triggeringpoint="stop"))

# stopTrigger = xosc.EntityTrigger(name = "tetetewtewtwe", delay = 0,
#                                         conditionedge = xosc.ConditionEdge.rising,
#                                         entitycondition = xosc.RelativeDistanceCondition(value = 5, entity = agentName, rule = "lessThan", dist_type = "cartesianDistance", freespace = "false"),
#                                         triggerentity = egoName, triggeringrule = "any",triggeringpoint = "stop")
sb.add_maneuver(egoManeuver, agentName)

### Create Scenario

In [9]:
# 实例化一个Scenario对象 xosc.scenario
sce = xosc.Scenario( 
    name="ITRI DEMO",
    author="YuCheng",
    parameters = paramdec,
    entities=entities,
    storyboard=sb,
    roadnetwork=road,
    catalog=catalog,
    osc_minor_version=0
)
prettyprint(sce.get_element())

sce.write_xml("C:/Users/USER/Documents/GitHub/esmini-demo/resources/xosc/hct_06.xosc")

<?xml version="1.0" encoding="utf-8"?>
<OpenSCENARIO xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="OpenScenario.xsd">
    <FileHeader description="ITRI DEMO" author="YuCheng" revMajor="1" revMinor="0" date="2024-06-26T17:18:52.232791"/>
    <ParameterDeclarations>
        <ParameterDeclaration name="EgoVehicle" parameterType="string" value="car_white"/>
        <ParameterDeclaration name="EgoSpeed" parameterType="double" value="60"/>
        <ParameterDeclaration name="Ego_S" parameterType="double" value="27"/>
        <ParameterDeclaration name="TargetVehicle" parameterType="string" value="car_red"/>
        <ParameterDeclaration name="TargetSpeed" parameterType="double" value="28"/>
        <ParameterDeclaration name="Target_S" parameterType="double" value="0"/>
    </ParameterDeclarations>
    <CatalogLocations>
        <VehicleCatalog>
            <Directory path="./Catalogs/Vehicles"/>
        </VehicleCatalog>
    </CatalogLocations>
    <Ro

In [10]:
# 实例化一个Scenario对象 xosc.scenario
sce = xosc.Scenario( 
    name="ITRI DEMO",
    author="YuCheng",
    parameters = paramdec,
    entities=entities,
    storyboard=sb,
    roadnetwork=road,
    catalog=catalog,
    osc_minor_version=0
)
# prettyprint(sce.get_element())
print("go")

#sce.write_xml("/home/yucheng/source_code/lssvip/resources/xosc/itri/testtest.xosc")

go


In [4]:
import os
from scenariogeneration import xosc, prettyprint, ScenarioGenerator


speed_of_outer_car = 20
import os
from scenariogeneration import xosc, prettyprint, ScenarioGenerator


class Scenario(ScenarioGenerator):
    def __init__(self):
        # super().__init__()
        self.open_scenario_version = 2

    def scenario(self, **kwargs):
        ### create catalogs
        catalog = xosc.Catalog()
        catalog.add_catalog("VehicleCatalog", "../xosc/Catalogs/Vehicles")

        ### create road
        road = xosc.RoadNetwork(
            roadfile="../xodr/e6mini.xodr", scenegraph="../models/e6mini.osgb"
        )

        ### create parameters
        paramdec = xosc.ParameterDeclarations()

        ## create entities

        egoname = "Ego"
        redname = "Target1"
        yelname = "Target2"

        entities = xosc.Entities()
        entities.add_scenario_object(
            egoname, xosc.CatalogReference("VehicleCatalog", "car_white")
        )
        entities.add_scenario_object(
            redname, xosc.CatalogReference("VehicleCatalog", "car_red")
        )
        entities.add_scenario_object(
            yelname, xosc.CatalogReference("VehicleCatalog", "car_yellow")
        )

        ### create init

        init = xosc.Init()
        step_time = xosc.TransitionDynamics(
            xosc.DynamicsShapes.step, xosc.DynamicsDimension.time, 1
        )

        init.add_init_action(egoname, xosc.AbsoluteSpeedAction(30, step_time))
        init.add_init_action(
            egoname, xosc.TeleportAction(xosc.LanePosition(25, 0, -3, 0))
        )
        init.add_init_action(redname, xosc.AbsoluteSpeedAction(40, step_time))
        init.add_init_action(
            redname, xosc.TeleportAction(xosc.LanePosition(15, 0, -2, 0))
        )
        init.add_init_action(yelname, xosc.AbsoluteSpeedAction(30, step_time))
        init.add_init_action(
            yelname, xosc.TeleportAction(xosc.LanePosition(35, 0, -4, 0))
        )

        ### create an event for the red car

        r_trigcond = xosc.TimeHeadwayCondition(redname, 0.1, xosc.Rule.greaterThan)
        r_trigger = xosc.EntityTrigger(
            "redtrigger", 0.2, xosc.ConditionEdge.rising, r_trigcond, egoname
        )
        r_event = xosc.Event("first_lane_change", xosc.Priority.overwrite)
        r_event.add_trigger(r_trigger)
        r_event.add_action(
            "lane_change_red",
            xosc.AbsoluteLaneChangeAction(
                -4,
                xosc.TransitionDynamics(
                    xosc.DynamicsShapes.sinusoidal, xosc.DynamicsDimension.time, 4
                ),
            ),
        )

        ## create the act for the red car
        r_man = xosc.Maneuver("red_maneuver")
        r_man.add_event(r_event)

        r_mangr = xosc.ManeuverGroup("mangroup_red")
        r_mangr.add_actor(redname)
        r_mangr.add_maneuver(r_man)

        act = xosc.Act(
            "red_act",
            xosc.ValueTrigger(
                "starttrigger",
                0,
                xosc.ConditionEdge.rising,
                xosc.SimulationTimeCondition(0, xosc.Rule.greaterThan),
            ),
        )
        act.add_maneuver_group(r_mangr)

        ## create an event for the yellow car

        y_trigcond = xosc.TimeHeadwayCondition(redname, 0.5, xosc.Rule.greaterThan)
        y_trigger = xosc.EntityTrigger(
            "yellow_trigger", 0, xosc.ConditionEdge.rising, y_trigcond, yelname
        )

        y_event = xosc.Event("yellow_lanechange", xosc.Priority.overwrite)
        y_event.add_trigger(y_trigger)

        y_event.add_action(
            "lane_change_yellow",
            xosc.AbsoluteLaneChangeAction(
                -3,
                xosc.TransitionDynamics(
                    xosc.DynamicsShapes.sinusoidal, xosc.DynamicsDimension.time, 2
                ),
            ),
        )

        ## create the act for the yellow car
        y_man = xosc.Maneuver("yellow_maneuver")
        y_man.add_event(y_event)

        y_mangr = xosc.ManeuverGroup("yellow_mangroup")
        y_mangr.add_actor(yelname)
        y_mangr.add_maneuver(y_man)
        y_starttrigger = xosc.ValueTrigger(
            "starttrigger",
            0,
            xosc.ConditionEdge.rising,
            xosc.SimulationTimeCondition(0, xosc.Rule.greaterThan),
        )
        # y_act = xosc.Act('my_act',y_starttrigger)
        act.add_maneuver_group(y_mangr)

        ## create the story

        story = xosc.Story("mystory")
        story.add_act(act)
        # story.add_act(y_act)

        ## create the storyboard
        sb = xosc.StoryBoard(
            init,
            xosc.ValueTrigger(
                "stop_simulation",
                0,
                xosc.ConditionEdge.rising,
                xosc.SimulationTimeCondition(10, xosc.Rule.greaterThan),
                "stop",
            ),
        )
        sb.add_story(story)

        ## create the scenario
        sce = xosc.Scenario(
            "adaptspeed_example",
            "Mandolin",
            paramdec,
            entities=entities,
            storyboard=sb,
            roadnetwork=road,
            catalog=catalog,
            osc_minor_version=self.open_scenario_version,
        )

        return sce


if __name__ == "__main__":
    sce = Scenario()
    # Print the resulting xml
    prettyprint(sce.scenario().get_element())

    # write the OpenSCENARIO file as xosc using current script name
    # sce.generate(".")

    # uncomment the following lines to display the scenario using esmini
    # from scenariogeneration import esmini
    # esmini(sce,os.path.join('esmini'))

<?xml version="1.0" encoding="utf-8"?>
<OpenSCENARIO xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="OpenScenario.xsd">
    <FileHeader description="adaptspeed_example" author="Mandolin" revMajor="1" revMinor="2" date="2024-07-02T18:23:16.119333"/>
    <CatalogLocations>
        <VehicleCatalog>
            <Directory path="../xosc/Catalogs/Vehicles"/>
        </VehicleCatalog>
    </CatalogLocations>
    <RoadNetwork>
        <LogicFile filepath="../xodr/e6mini.xodr"/>
        <SceneGraphFile filepath="../models/e6mini.osgb"/>
    </RoadNetwork>
    <Entities>
        <ScenarioObject name="Ego">
            <CatalogReference catalogName="VehicleCatalog" entryName="car_white"/>
        </ScenarioObject>
        <ScenarioObject name="Target1">
            <CatalogReference catalogName="VehicleCatalog" entryName="car_red"/>
        </ScenarioObject>
        <ScenarioObject name="Target2">
            <CatalogReference catalogName="VehicleCatalog" entr